In [1]:
%pip install -q pyomo
!apt-get install -y -qq coinor-cbc

Note: you may need to restart the kernel to use updated packages.
E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?


In [2]:
from pyomo.environ import *
import math
n=3                         #Number of sets
alpha=1                     #Alpha
beta=1                      #Beta
b=1000                      #This is the budget
model=ConcreteModel()

In [3]:
model.xN = Var(range(n+1),domain= NonNegativeReals)            #Money bet in i sets on N for i in (1,2,3) and for match if i=0
model.xJ= Var(range(n+1),domain= NonNegativeReals)             #Money bet in i sets on J for i in (1,2,3) and for match if i=0
model.t=Var(range(n+1),domain=Binary)                          #t[0] is for match, and t[1,2,3] for sets. This represents the worst case profit

In [4]:
model.obj = Objective(expr = b+sum(model.t[i] for i in range(n+1)), sense = maximize)

In [5]:
model.constraints = ConstraintList()
model.constraints.add(expr=model.t[0]<=model.xN[0]/beta-model.xJ[0])
model.constraints.add(expr=model.t[0]<=model.xJ[0]*beta-model.xN[0])
for i in range(1,n+1):
    model.constraints.add(expr=model.t[i]<=model.xN[i]/alpha-model.xJ[i])
    model.constraints.add(expr=model.t[i]<=model.xJ[i]*alpha-model.xN[i])

In [6]:
model.constraints.add(expr=b>=sum(model.xJ[i] for i in range (n+1))+sum(model.xN[i] for i in range (n+1)))

In [7]:
for i in range(n+1):
    model.constraints.add(expr=model.xJ[i]>=0)
    model.constraints.add(expr=model.xN[i]>=0)

In [8]:
opt = SolverFactory('cbc')
result = opt.solve(model, tee = True)
print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)

Welcome to the CBC MILP Solver 
Version: 2.10.8 
Build Date: May  9 2022 

command line - /bin/cbc -printingOptions all -import /tmp/tmpy84_y_e8.pyomo.lp -stat=1 -solve -solu /tmp/tmpy84_y_e8.pyomo.soln (default strategy 1)
Option for printingOptions changed from normal to all
 CoinLpIO::readLp(): Maximization problem reformulated as minimization
Coin0009I Switching back to maximization to get correct duals etc
Presolve 9 (-8) rows, 12 (-1) columns and 32 (-8) elements
Statistics for presolved model
Original problem has 4 integers (4 of which binary)
Presolved problem has 4 integers (4 of which binary)
==== 8 zero objective 2 different
8 variables have objective of -0
4 variables have objective of 1
==== absolute objective values 2 different
8 variables have objective of 0
4 variables have objective of 1
==== for integers 0 zero objective 1 different
4 variables have objective of 1
==== for integers absolute objective values 1 different
4 variables have objective of 1
===== end objecti

In [9]:
print('Optimal money', model.obj())

Optimal money 1000.0
